In [3]:
# 필수 패키지 설치
!pip install transformers datasets

In [4]:
# 1. HuggingFace 모델 불러와서 감정 분석하기 (pipeline 방식)
from transformers import pipeline

# 1) 감정 분석 전용 pipeline 생성
sentiment_classifier = pipeline("sentiment-analysis")

# 2) 테스트 문장 입력
texts = [
    "I really love this product! IT works perfectly!",
    "This is the worst expreience I've ever had."
]

# 3) 결과 출력
results = sentiment_classifier(texts)
for text, result in zip(texts, results):
    print(f"입력: {text}")
    print(f"예측 결과: {result}\n")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


입력: I really love this product! IT works perfectly!
예측 결과: {'label': 'POSITIVE', 'score': 0.9998819828033447}

입력: This is the worst expreience I've ever had.
예측 결과: {'label': 'NEGATIVE', 'score': 0.9997898936271667}



In [7]:
# 2) 모델 + 토크나이저 직접 불러와서 감정 분석하기 (로우레벨 방식)
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
import torch

# 1) 모델 이름 지정
MODEL_NAME = "distilbert-base-uncased-finetuned-sst-2-english"

# 2) 토크나이저 & 모델 로딩
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

# 3) 입력 문장
text = "I don't like this movie. It's boring."

# 4) 토큰화 (문장을 텐서로 변환)
inputs = tokenizer(text, return_tensors="pt")

# 5) 모델 forward pass (logits 출력)
outputs = model(**inputs)

# 6) softmax로 감정 확률 계산
probs = torch.nn.functional.softmax(outputs.logits, dim=-1)

# 7) 레이블 매핑 ( 0=NEGATIVE, 1=POSITIVE )
labels = ['NEGATIVE', 'POSITIVE']
pred_label = labels[torch.argmax(probs)]

print(f"입력 문장: {text}")
print(f"예측 감정: {pred_label}")
print(f"확률: {probs.tolist()}")

입력 문장: I don't like this movie. It's boring.
예측 감정: NEGATIVE
확률: [[0.9994487166404724, 0.0005512956413440406]]


AutoModelForSequenceClassification : HuggingFace Transformers 라이브러리에서, "문장 단위 분류를 수행할 수 있도록 사전 정의된 모델 클래스를 자동으로 불러오는 역할"

### 데이터셋 불러와서 사전 학습된 모델로 감정 분류 해보기

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from datasets import load_dataset


# 1) 감정 분석 모델 지정
MODEL_NAME = "distilbert-base-uncased-finetuned-sst-2-english"

# 2) 데이터셋 로드 (SST-2, 영어 감정 분석 데이터)
dataset = load_dataset("glue","sst2")
test_data = dataset["validation"] # 테스트용 데이터 split

# 3) 토크나이저 & 모델 로딩
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

# 4) 테스트 데이터의 첫 10개 문장 예측 실행
for i in range(10) :
    text = test_data[i]["sentence"]
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    probs = torch.nn.functional.softmax(outputs.logits, dim = -1 )
    labels = ["NEGATIVE", "POSITIVE"]
    pred_label = labels[torch.argmax(probs)]

    print(f"문장 : {text}")
    print(f"예측 감정 : {pred_label}, 확률 : {probs.tolist()}\n")

문장 : it 's a charming and often affecting journey . 
예측 감정 : POSITIVE, 확률 : [[0.00011603943858062848, 0.9998838901519775]]

문장 : unflinchingly bleak and desperate 
예측 감정 : NEGATIVE, 확률 : [[0.998969554901123, 0.0010304978350177407]]

문장 : allows us to hope that nolan is poised to embark a major career as a commercial yet inventive filmmaker . 
예측 감정 : POSITIVE, 확률 : [[0.00035998859675601125, 0.9996399879455566]]

문장 : the acting , costumes , music , cinematography and sound are all astounding given the production 's austere locales . 
예측 감정 : POSITIVE, 확률 : [[0.0003007006598636508, 0.9996993541717529]]

문장 : it 's slow -- very , very slow . 
예측 감정 : NEGATIVE, 확률 : [[0.9996892213821411, 0.00031081135966815054]]

문장 : although laced with humor and a few fanciful touches , the film is a refreshingly serious look at young women . 
예측 감정 : POSITIVE, 확률 : [[0.0001709073258098215, 0.9998290538787842]]

문장 : a sometimes tedious film . 
예측 감정 : NEGATIVE, 확률 : [[0.999504566192627, 0.0004954662872